In [1]:
# استدعاء المكتبات اللي هنحتاجها
import pandas as pd  # لتحميل وتحليل الداتا
import numpy as np  # للعمليات الرياضية
from sklearn.model_selection import train_test_split, GridSearchCV  # لتقسيم الداتا وتحسين النماذج
from sklearn.preprocessing import StandardScaler, LabelEncoder  # لتظبيط الداتا
from sklearn.linear_model import LogisticRegression  # نموذج بسيط
from sklearn.ensemble import RandomForestClassifier  # نموذج قوي
from xgboost import XGBClassifier  # نموذج متقدم
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report  # لقياس الأداء

# الخطوة 1: تحميل الداتاسيت
# لو بتستخدمي Google Colab، فعّلي السطرين دول الأول عشان ترفعي الملف
# from google.colab import files
# uploaded = files.upload()
data = pd.read_csv('/content/Deep_clean.csv')  # تحميل ملف الداتا

# الخطوة 2: إنشاء عمود الـ Churn
# العميل هيسيب لو رضا العميل <= 4 أو مش في برنامج الولاء
data['Churn'] = ((data['Customer_Satisfaction'] <= 4) | (data['Customer_Loyalty_Program_Member'] == False)).astype(int)

# الخطوة 3: تحضير الداتا
# التأكد من القيم الناقصة
print("القيم الناقصة:\n", data.isnull().sum())
# لو فيه قيم ناقصة، ممكن نعبيها، بس الداتاسيت نضيفة فما فيهاش مشاكل

# تحديد الأعمدة النصية والرقمية
categorical_cols = ['Gender', 'Income_Level', 'Marital_Status', 'Education_Level', 'Occupation',
                    'Location', 'Purchase_Category', 'Purchase_Channel', 'Brand_Loyalty',
                    'Social_Media_Influence', 'Discount_Sensitivity', 'Engagement_with_Ads',
                    'Device_Used_for_Shopping', 'Payment_Method', 'Discount_Used',
                    'Customer_Loyalty_Program_Member', 'Purchase_Intent', 'Shipping_Preference']
numerical_cols = ['Age', 'Purchase_Amount', 'Frequency_of_Purchase', 'Product_Rating',
                 'Time_Spent_on_Product_Research(hours)', 'Return_Rate', 'Customer_Satisfaction',
                 'Time_to_Decision']

# تحويل الأعمدة النصية لأرقام
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()  # أداة بتحوّل النصوص لأرقام
    data[col] = le.fit_transform(data[col])  # تحويل العمود
    label_encoders[col] = le  # حفظ الأداة لو احتجناها

# تظبيط الأعمدة الرقمية (جعلها في نطاق موحد)
scaler = StandardScaler()  # أداة لتظبيط الأرقام
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])  # تطبيق التظبيط

# الخطوة 4: تحديد الـ Features (X) والـ Target (y)
X = data.drop(['Customer_ID', 'Time_of_Purchase', 'Churn'], axis=1)  # إزالة الأعمدة اللي مش هنستخدمها
y = data['Churn']  # الهدف (Churn)

# الخطوة 5: تقسيم الداتا
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # 80% تدريب، 20% اختبار

# الخطوة 6: تدريب وتقييم النماذج
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),  # نموذج بسيط
    'Random Forest': RandomForestClassifier(random_state=42),  # نموذج قوي
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)  # نموذج متقدم
}

# قاموس لتخزين النتايج
results = {}

for name, model in models.items():
    # تدريب النموذج
    model.fit(X_train, y_train)

    # التنبؤ على داتا الاختبار
    y_pred = model.predict(X_test)

    # قياس الأداء
    accuracy = accuracy_score(y_test, y_pred)  # الدقة
    precision = precision_score(y_test, y_pred)  # الدقة في التنبؤ بالـ Churn
    recall = recall_score(y_test, y_pred)  # القدرة على إيجاد الـ Churn
    f1 = f1_score(y_test, y_pred)  # مزيج بين Precision و Recall

    # تخزين النتايج
    results[name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1
    }

    # طباعة تقرير مفصل
    print(f"\nنتايج {name}:")
    print(classification_report(y_test, y_pred))

# الخطوة 7: تحسين نموذج Random Forest
param_grid = {
    'n_estimators': [100, 200],  # عدد الأشجار
    'max_depth': [10, 20, None],  # العمق
    'min_samples_split': [2, 5]  # الحد الأدنى للتقسيم
}
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='f1')
grid_search.fit(X_train, y_train)

# أفضل الإعدادات
print("\nأفضل إعدادات لـ Random Forest:", grid_search.best_params_)
print("أفضل F1-Score:", grid_search.best_score_)

# تدريب النموذج المحسن
best_rf = grid_search.best_estimator_
y_pred_rf = best_rf.predict(X_test)
print("\nنتايج Random Forest المحسن:")
print(classification_report(y_test, y_pred_rf))

# الخطوة 8: تلخيص النتايج
print("\nملخص أداء النماذج:")
for name, metrics in results.items():
    print(f"{name}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")

القيم الناقصة:
 Customer_ID                              0
Age                                      0
Gender                                   0
Income_Level                             0
Marital_Status                           0
Education_Level                          0
Occupation                               0
Location                                 0
Purchase_Category                        0
Purchase_Amount                          0
Frequency_of_Purchase                    0
Purchase_Channel                         0
Brand_Loyalty                            0
Product_Rating                           0
Time_Spent_on_Product_Research(hours)    0
Social_Media_Influence                   0
Discount_Sensitivity                     0
Return_Rate                              0
Customer_Satisfaction                    0
Engagement_with_Ads                      0
Device_Used_for_Shopping                 0
Payment_Method                           0
Time_of_Purchase                      

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [04:27:38] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



أفضل إعدادات لـ Random Forest: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 100}
أفضل F1-Score: 1.0

نتايج Random Forest المحسن:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        67
           1       1.00      1.00      1.00       133

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200


ملخص أداء النماذج:
Logistic Regression:
  Accuracy: 0.9850
  Precision: 0.9779
  Recall: 1.0000
  F1-Score: 0.9888
Random Forest:
  Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1-Score: 1.0000
XGBoost:
  Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1-Score: 1.0000


In [2]:
# كود عشان نعرف أهم العوامل (Feature Importance)
importances = best_rf.feature_importances_
feature_names = X.columns
for name, importance in zip(feature_names, importances):
    print(f"{name}: {importance:.4f}")

Age: 0.0222
Gender: 0.0109
Income_Level: 0.0039
Marital_Status: 0.0111
Education_Level: 0.0078
Occupation: 0.0065
Location: 0.0301
Purchase_Category: 0.0205
Purchase_Amount: 0.0339
Frequency_of_Purchase: 0.0177
Purchase_Channel: 0.0074
Brand_Loyalty: 0.0116
Product_Rating: 0.0091
Time_Spent_on_Product_Research(hours): 0.0086
Social_Media_Influence: 0.0090
Discount_Sensitivity: 0.0067
Return_Rate: 0.0072
Customer_Satisfaction: 0.3116
Engagement_with_Ads: 0.0063
Device_Used_for_Shopping: 0.0082
Payment_Method: 0.0137
Discount_Used: 0.0045
Customer_Loyalty_Program_Member: 0.3944
Purchase_Intent: 0.0110
Shipping_Preference: 0.0064
Time_to_Decision: 0.0197


In [3]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(best_rf, X, y, cv=5, scoring='f1')
print(f"Cross-Validation F1-Scores: {scores}")
print(f"متوسط F1-Score: {scores.mean():.4f}")

Cross-Validation F1-Scores: [1. 1. 1. 1. 1.]
متوسط F1-Score: 1.0000


In [4]:
import joblib

# حفظ نموذج Logistic Regression
joblib.dump(models['Logistic Regression'], 'logistic_regression_model.pkl')

# حفظ نموذج Random Forest (المحسن)
joblib.dump(best_rf, 'random_forest_model.pkl')

# حفظ نموذج XGBoost
joblib.dump(models['XGBoost'], 'xgboost_model.pkl')

['xgboost_model.pkl']